In [1]:
!pip -q install ftfy regex tqdm
!pip -q install git+https://github.com/openai/CLIP.git
!pip -q install pytorch-metric-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.7 MB/s eta 0:00

## Load dataset


In [2]:
#train-val-test: 70:15:15
!gdown 1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I
!unzip -q data_cifar10_ent_split.zip

Downloading...
From (original): https://drive.google.com/uc?id=1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I
From (redirected): https://drive.google.com/uc?id=1u9XcSIB4-UMmgSGa_or2B3bBbgiNZe5I&confirm=t&uuid=f807294f-eb81-40db-b0c4-554376971f6f
To: /kaggle/working/data_cifar10_ent_split.zip
100%|██████████████████████████████████████| 1.06G/1.06G [00:10<00:00, 99.3MB/s]


In [4]:

import os
import torch
import clip
from PIL import Image
import torch.nn.functional as F
import random
from sklearn.model_selection import train_test_split
import numpy as np

import torch.nn as nn
from pytorch_metric_learning import losses
from random import shuffle
from tqdm import tqdm
import torch.nn.init as init

from torch.optim.lr_scheduler import ReduceLROnPlateau

## Prepare Dataset

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load CLIP model (ViT-B/32)
model, preprocess = clip.load("ViT-B/32", device=device)

img_dir = "./train"
embeddings = {}

def load_images_recursively(folder):
    '''
    Embed all images in the training data and return them in a dictionary
    Return:
                embeddings = {
                    "img_name": embedding_tensor,
                    ...
                }
    '''
    embeddings = {}
    for root, dirs, files in os.walk(folder):
        for fn in files:
            if fn.lower().endswith((".png", ".jpg", ".jpeg")):
                path = os.path.join(root, fn)
                try:
                    image = preprocess(Image.open(path)).unsqueeze(0).to(device)
                    with torch.no_grad():
                        emb = model.encode_image(image)
                        emb = emb / emb.norm(dim=-1, keepdim=True)
                    rel_path = os.path.relpath(path, img_dir)
                    embeddings[rel_path] = emb.squeeze(0).cpu()
                except Exception as e:
                    print(f"Failed to load {path}: {e}")
    return embeddings

embeddings = load_images_recursively(img_dir)
print(f"Total images loaded: {len(embeddings)}")

100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 49.9MiB/s]


Total images loaded: 1298


In [7]:

def create_cls_map(root_dir):
    """
    Create a mapping from relative image path to its class name.

    Args:
        root_dir (str): Root directory containing class-named subfolders with images.

    Returns:
        dict: {
            "class_name/image_name.jpg": "class_name",
            ...
        }
    """
    cls_map = {}
    for cls_name in os.listdir(root_dir):
        cls_path = os.path.join(root_dir, cls_name)
        if not os.path.isdir(cls_path):
            continue
        for img_name in os.listdir(cls_path):
            if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                rel_path = f"{cls_name}/{img_name}"  # or use os.path.join and os.path.relpath for robustness
                cls_map[rel_path] = cls_name
    return cls_map
root_dir = img_dir
cls_map = create_cls_map(root_dir)
print(f"Created cls_map for {len(cls_map)} images")

Created cls_map for 1298 images


In [34]:
import torch.nn.functional as F

def generate_parent_child(cls_map, embeddings, num_children=3, verbose=False):
    """
    Generate (parent, top-k children) pairs based on cosine similarity within the same class.

    Returns:
        - parent_child: dict of {img_name: [child1, child2, ...]}
        - class_to_imgs: dict of {class_name: [img_name1, img_name2, ...]}
    """
    class_to_imgs = {}
    for img_name, cls in cls_map.items():
        class_to_imgs.setdefault(cls, []).append(img_name)

    parent_child = {}
    skipped_classes = 0

    for cls, img_list in class_to_imgs.items():
        valid_imgs = [i for i in img_list if i in embeddings]
        if len(valid_imgs) < 2:
            skipped_classes += 1
            continue  # Không đủ ảnh để tạo parent-child

        for img_name in valid_imgs:
            anchor_emb = embeddings[img_name].unsqueeze(0)

            others = [i for i in valid_imgs if i != img_name]
            if not others:
                continue

            other_embs = torch.stack([embeddings[i] for i in others])

            sim_scores = F.cosine_similarity(anchor_emb, other_embs)
            topk = min(num_children, len(others))
            topk_indices = torch.topk(sim_scores, k=topk).indices

            children = [others[i] for i in topk_indices.tolist()]
            parent_child[img_name] = children

    if verbose:
        print(f"✅ Generated parent-child pairs: {len(parent_child)}")
        print(f"⚠️ Skipped classes with <2 valid embeddings: {skipped_classes}")

    return parent_child, class_to_imgs

parent_child, class_to_imgs = generate_parent_child(cls_map, embeddings, num_children=3)
print(f"Generated parent-child pairs for {len(parent_child)} images")

Generated parent-child pairs for 1298 images


In [10]:
import torch
import torch.nn.functional as F
import random

# Set random seeds for reproducibility
random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Define class-wise incompatible (negative) relationships
negative_map = {
    "nose-right": ["nose-left"],
    "nose-left": ["nose-right"],
    "ear-right": ["ear-left"],
    "ear-left": ["ear-right"],
    "vc-open": ["vc-closed", "throat"],
    "vc-closed": ["vc-open"],
    "throat": ["vc-open", "vc-closed"],
}

# --- Function definitions ---

def get_hard_negative(anchor_name, anchor_emb, anchor_cls, class_to_imgs, embeddings, negative_map):
    """
    Select the hardest negative image (most similar but from a different class).
    Priority is given to predefined negative classes.
    """
    neg_classes = negative_map.get(anchor_cls, None)

    if not neg_classes:
        neg_cls_candidates = [c for c in class_to_imgs if c != anchor_cls]
        if not neg_cls_candidates:
            return None
        neg_cls = random.choice(neg_cls_candidates)
        neg_imgs = class_to_imgs[neg_cls]
    else:
        neg_imgs = []
        for neg_cls in neg_classes:
            neg_imgs.extend(class_to_imgs.get(neg_cls, []))
        if not neg_imgs:
            return None

    anchor_emb = anchor_emb.unsqueeze(0)
    max_sim = -1
    hard_neg = None
    for neg_img in neg_imgs:
        if neg_img not in embeddings:
            continue
        neg_emb = embeddings[neg_img].unsqueeze(0)
        sim = F.cosine_similarity(anchor_emb, neg_emb).item()
        if sim > max_sim:
            max_sim = sim
            hard_neg = neg_img

    return hard_neg


def filter_parent_child(parent_child_full, embeddings_subset):
    """
    Filter parent-child mapping để giữ các cặp có mặt trong embeddings hiện tại.
    """
    parent_child_filtered = {}
    valid_imgs = set(embeddings_subset.keys())
    for parent, children in parent_child_full.items():
        if parent in valid_imgs:
            filtered = [c for c in children if c in valid_imgs]
            if filtered:
                parent_child_filtered[parent] = filtered
    return parent_child_filtered


def create_triplets(embeddings, cls_map, parent_child, class_to_imgs):
    """
    Tạo các bộ 3 (anchor, positive, negative) từ tập embeddings hiện tại.
    """
    triplets = []
    for anchor_name, anchor_emb in embeddings.items():
        if anchor_name not in parent_child or len(parent_child[anchor_name]) == 0:
            continue
        anchor_cls = cls_map[anchor_name]
        pos_name = parent_child[anchor_name][0]
        neg_name = get_hard_negative(anchor_name, anchor_emb, anchor_cls, class_to_imgs, embeddings, negative_map)

        if neg_name is not None:
            triplets.append((
                anchor_emb,
                embeddings[pos_name],
                embeddings[neg_name],
                anchor_name,
                neg_name
            ))

    print(f"Generated {len(triplets)} triplets.")
    return triplets


def get_triplet_batch(triplets, batch_size=32):
    if len(triplets) < batch_size:
        return None
    batch = random.sample(triplets, batch_size)
    anchor = torch.stack([t[0] for t in batch])
    positive = torch.stack([t[1] for t in batch])
    negative = torch.stack([t[2] for t in batch])
    return anchor, positive, negative


parent_child = filter_parent_child(parent_child, embeddings)
triplets = create_triplets(embeddings, cls_map, parent_child, class_to_imgs)
batch = get_triplet_batch(triplets, batch_size=32)
if batch:
    anchor, positive, negative = batch
    print(anchor.shape, positive.shape, negative.shape)
else:
    print("Không đủ triplet để tạo batch.")


Generated 1298 triplets.
torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32, 512])


## Training

In [12]:
embed_dim = 512
class GaussianFourierProjection(nn.Module):
    def __init__(self, embed_dim, scale=10.0):
        super().__init__()
        # Fixed random weights for projecting scalar t to higher frequency space
        self.W = nn.Parameter(torch.randn(1, embed_dim // 2) * scale, requires_grad=False)

    def forward(self, t):
        # Ensure t has shape [B, 1]
        if t.ndim == 1:
            t = t.unsqueeze(-1)
        proj = t * self.W  # Shape: [B, D/2]
        # Return sinusoidal and cosinusoidal projection: [sin(tW), cos(tW)] → Shape: [B, D]
        return torch.cat([torch.sin(proj), torch.cos(proj)], dim=-1)

class VectorField(nn.Module):
    def __init__(self, dim, t_dim=32, hidden_dim=256, n_heads=4, dropout_prob=0.1):
        super().__init__()
        self.x_norm = nn.LayerNorm(dim)  # Normalize input embeddings
        self.time_encoder = GaussianFourierProjection(t_dim)  # Time embedding module
        self.dropout = nn.Dropout(dropout_prob)

        # Create multiple independent heads (like a lightweight transformer block)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(dim + t_dim, hidden_dim),     # Project input + time
                nn.LayerNorm(hidden_dim),
                nn.SiLU(),                               # Activation: SiLU 
                nn.Dropout(dropout_prob),
                nn.Linear(hidden_dim, dim)              # Back to original embedding dimension
            ) for _ in range(n_heads)
        ])

        self.res_weight = nn.Parameter(torch.tensor(1.0))  # Learnable residual scaling
        self.out_norm = nn.LayerNorm(dim)  # Final normalization (not applied directly here)
        self.initialize_weights()

    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Use Kaiming initialization (good for ReLU/SiLU)
                init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    init.zeros_(m.bias)


    def forward(self, x, t):
        # Handle scalar or 1D tensor time input → ensure shape [B, 1]
        if not isinstance(t, torch.Tensor):
            t = torch.full((x.shape[0], 1), t, device=x.device)
        elif t.ndim == 0:
            t = t.expand(x.shape[0], 1)
        elif t.ndim == 1:
            t = t.unsqueeze(-1)

        x_normed = self.x_norm(x)                     # Normalize input
        t_encoded = self.time_encoder(t.to(x.device)) # Encode time t
        inp = torch.cat([x_normed, t_encoded], dim=-1)  # Concatenate along feature dim

        # Pass through each head and average their outputs
        head_outs = [head(inp) for head in self.heads]
        out = torch.mean(torch.stack(head_outs), dim=0)

        # Add residual connection scaled by learnable weight
        return out + self.res_weight * x

In [43]:
def euler_integration(x0, vf, steps=10):
    """
    Args:
        x0: initial embeddings [B, D]
        vf: vector field model (takes in x and t, returns dx/dt)
        steps: number of integration steps
    Returns:
        Transformed embeddings x(T)
    """
    dt = 1.0 / steps
    x = x0
    for i in range(steps):
        t = i * dt
        k1 = vf(x, t)
        k2 = vf(x + 0.5 * dt * k1, t + 0.5 * dt)
        k3 = vf(x + 0.5 * dt * k2, t + 0.5 * dt)
        k4 = vf(x + dt * k3, t + dt)
        x = x + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    return x

    #return correct / total if total > 0 else 0.0
def compute_triplet_loss(model, triplets, loss_func, cls_map, steps=10):
    """
       Compute loss using MultiSimilarityLoss
       It takes transformed embeddings + integer labels
   
    """
    model.eval()
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for i in range(0, len(triplets), 64):
            batch = triplets[i : i + 64]
            if len(batch) == 0:
                continue

            batch_embeddings, batch_labels = get_embeddings_labels_from_triplets(batch, cls_map, class2idx)
            if batch_embeddings is None or len(batch_embeddings) == 0:
                continue

            batch_embeddings = batch_embeddings.to(device).float()
            batch_labels = batch_labels.to(device).long()

            pred_embeddings = euler_integration(batch_embeddings, model, steps=steps)
            loss = loss_func(pred_embeddings, batch_labels)

            total_loss += loss.item()
            num_batches += 1

    return total_loss / max(1, num_batches)

def get_embeddings_labels_from_triplets(triplets_batch, cls_map, class2idx):
    '''
        Convert list of triplets into tensors for model input.
    '''
    embeddings = []
    labels = []

    for anchor_emb, positive_emb, negative_emb, anchor_name, negative_name in triplets_batch:
        anchor_cls = cls_map[anchor_name]
        negative_cls = cls_map[negative_name]

        embeddings.extend([anchor_emb, positive_emb, negative_emb])
        labels.extend([
            class2idx[anchor_cls],
            class2idx[anchor_cls],
            class2idx[negative_cls]
        ])

    embeddings_tensor = torch.stack(embeddings)
    labels_tensor = torch.tensor(labels)
    return embeddings_tensor, labels_tensor

In [36]:
root_dir_val = "./val"

def standardize_keys(d):
    return {
        os.path.join(os.path.basename(os.path.dirname(k)), os.path.basename(k)): v
        for k, v in d.items()
    }

# 1. Load embeddings
embeddings_val = load_images_recursively(root_dir_val)
print(f" Total images loaded: {len(embeddings_val)}")

# 2. Load cls_map
cls_map_val = create_cls_map(root_dir_val)
print(f" Created cls_map for {len(cls_map_val)} images")

# 3.  Standardize keys
embeddings_val = standardize_keys(embeddings_val)
cls_map_val = standardize_keys(cls_map_val)

# 4. Generate parent-child
parent_child_val, class_to_imgs_val = generate_parent_child(cls_map_val, embeddings_val, num_children=3, verbose=True)

# 5. Filter parent-child
parent_child_val = filter_parent_child(parent_child_val, embeddings_val)

# 6. Create triplets
triplets_val = create_triplets(embeddings_val, cls_map_val, parent_child_val, class_to_imgs_val)
print(f" Total triplets generated: {len(triplets_val)}")

# 7. Get batch
batch_val = get_triplet_batch(triplets_val, batch_size=32)
if batch_val:
    anchor_val, positive_val, negative_val = batch_val
    print(" Batch shapes:", anchor_val.shape, positive_val.shape, negative_val.shape)
else:
    print(" Không đủ triplet để tạo batch.")


 Total images loaded: 275
 Created cls_map for 275 images
✅ Generated parent-child pairs: 275
⚠️ Skipped classes with <2 valid embeddings: 0
Generated 275 triplets.
 Total triplets generated: 275
 Batch shapes: torch.Size([32, 512]) torch.Size([32, 512]) torch.Size([32, 512])


In [37]:
sample_cls_key = list(cls_map_val.keys())[0]
sample_emb_key = list(embeddings_val.keys())[0]
print("🔍 Sample key từ cls_map:", sample_cls_key)
print("🔍 Sample key từ embeddings:", sample_emb_key)


🔍 Sample key từ cls_map: vc-open/23146955_230624145953203831_954_image04.png
🔍 Sample key từ embeddings: vc-open/23146955_230624145953203831_954_image04.png


In [38]:
@torch.no_grad()
def compute_recall_at_1_embedded(embeddings, cls_map, type_map, vf, device="cuda", steps=10):
    """
    embeddings: dict tên ảnh → vector trước khi qua VF
    cls_map: dict tên ảnh → class (string)
    type_map: dict class → list các class cùng type
    vf: mô hình vector field (đã học)
    """
    keys = list(embeddings.keys())
    embedded_vectors = {}
    
    # Apply VF để lấy vector sau khi flow
    for k in keys:
        emb = embeddings[k].unsqueeze(0).to(device).float()
        emb = euler_integration(emb, vf, steps=steps)
        embedded_vectors[k] = emb.squeeze(0).cpu()  # chuyển lại về CPU để so cosine

    correct = 0
    total = 0

    for anchor_name in keys:
        anchor_emb = embedded_vectors[anchor_name].unsqueeze(0)
        anchor_cls = cls_map[anchor_name]
        anchor_type = set(type_map.get(anchor_cls, [])) | {anchor_cls}

        max_sim = -float("inf")
        top1_name = None

        for other_name in keys:
            if other_name == anchor_name:
                continue
            other_emb = embedded_vectors[other_name].unsqueeze(0)
            sim = F.cosine_similarity(anchor_emb, other_emb).item()
            if sim > max_sim:
                max_sim = sim
                top1_name = other_name

        if top1_name:
            pred_cls = cls_map[top1_name]
            if pred_cls in anchor_type:
                correct += 1
        total += 1

    recall = correct / total if total > 0 else 0.0
    return recall


In [46]:
#Define label mapping
class2idx = {'ear-left': 3,
 'ear-right': 2,
 'nose-left': 1,
 'nose-right': 0,
 'throat': 6,
 'vc-closed': 4,
 'vc-open': 5}

# Khởi tạo model, optimizer, loss
vf = VectorField(embed_dim).to(device).float()
optimizer = torch.optim.AdamW(vf.parameters(), lr=1e-4)
loss_func = losses.MultiSimilarityLoss()

# Hyperparameters cho scheduler và early stopping
warmup_epochs = 20
early_stop_patience = 40
scheduler_patience = 15
scheduler_factor = 0.8
delta = 5e-4

# LR Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=scheduler_factor, patience=scheduler_patience)

# Biến theo dõi
best_val_loss = float('inf')
best_train_loss = float('inf')
epochs_no_improve = 0

# Huấn luyện
epochs = 200
batch_size = 32

for epoch in tqdm(range(epochs)):
    vf.train()
    epoch_loss = 0.0
    num_batches = 0

    random.seed(42)
    random.shuffle(triplets)

    for i in range(0, len(triplets), batch_size):
        batch = triplets[i: i + batch_size]
        if not batch:
            continue

        optimizer.zero_grad()
        batch_embeddings, batch_labels = get_embeddings_labels_from_triplets(batch, cls_map, class2idx)

        if batch_embeddings is None or len(batch_embeddings) == 0:
            continue

        batch_embeddings = batch_embeddings.to(device).float()
        batch_labels = batch_labels.to(device).long()
        pred_embeddings = euler_integration(batch_embeddings, vf, steps=10)

        loss = loss_func(pred_embeddings, batch_labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(vf.parameters(), max_norm=5.0)
        optimizer.step()

        epoch_loss += loss.item()
        num_batches += 1

    # --- Evaluation ---
    vf.eval()
    val_loss = compute_triplet_loss(vf, triplets_val, loss_func,cls_map_val, steps=10)
    avg_train_loss = epoch_loss / max(1, num_batches)


    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f}")

    # --- LR Warmup & Scheduler ---
    old_lr = optimizer.param_groups[0]['lr']
    if epoch >= warmup_epochs:
        scheduler.step(val_loss)
    else:
        scheduler.step(float('inf'))

    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < old_lr:
        print(f"⚠️ LR reduced at epoch {epoch + 1} → {new_lr:.1e}")

    # --- Early Stopping ---
    if (val_loss < best_val_loss - delta) or (avg_train_loss < best_train_loss - delta):
        best_val_loss = min(val_loss, best_val_loss)
        best_train_loss = min(avg_train_loss, best_train_loss)
        epochs_no_improve = 0
        torch.save(vf.state_dict(), "best_vf.pt")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"⛔ Early stopping at epoch {epoch + 1}")
        break

  0%|          | 1/200 [00:18<59:51, 18.05s/it]

[Epoch 1] Train Loss: 1.4995


  1%|          | 2/200 [00:36<1:00:39, 18.38s/it]

[Epoch 2] Train Loss: 1.5001


  2%|▏         | 3/200 [00:54<1:00:00, 18.28s/it]

...

[Epoch 198] Train Loss: 0.9810
⚠️ LR reduced at epoch 198 → 6.9e-06


100%|█████████▉| 199/200 [1:01:23<00:18, 18.54s/it]

[Epoch 199] Train Loss: 0.9747


100%|██████████| 200/200 [1:01:43<00:00, 18.52s/it]

[Epoch 200] Train Loss: 0.9738


In [47]:
torch.save(vf.state_dict(), "vf_model.pth")

In [50]:
root_dir_test = "./test"

def standardize_keys(d):
    return {
        os.path.join(os.path.basename(os.path.dirname(k)), os.path.basename(k)): v
        for k, v in d.items()
    }

# 1. Load embeddings
embeddings_test = load_images_recursively(root_dir_test)
print(f" Total images loaded: {len(embeddings_test)}")

# 2. Load cls_map
cls_map_test = create_cls_map(root_dir_test)
print(f" Created cls_map for {len(cls_map_test)} images")

# 3.  Standardize keys
embeddings_test = standardize_keys(embeddings_test)
cls_map_test = standardize_keys(cls_map_test)



 Total images loaded: 284
 Created cls_map for 284 images


In [51]:
recall_val = compute_recall_at_1_embedded(embeddings_test, cls_map_test, type_map, vf, device=device)

In [52]:
recall_val

0.954225352112676